In [1]:
import pandas as pd

In [2]:
pd.__version__

'0.25.1'

# Extract happiness CSV dataset into DataFrames

In [38]:
happiness_df = pd.read_csv('happiness.csv')
happiness_df.head()


,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


# Renaming the happiness data columns

In [46]:
happiness_cols =['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high','Whisker.low', 'Economy..GDP.per.Capita.', 'Family', 'Health..Life.Expectancy.', 'Freedom', 'Generosity', 'Trust..Government.Corruption.', 'Dystopia.Residual']

happiness_transformed= happiness_df[happiness_cols].copy()


happiness_transformed = happiness_transformed.rename(columns={"Country": "country",
                                                         "Happiness.Rank": "happiness_rank",
                                                         "Happiness.Score": "happiness_score",
                                                         "Whisker.high": "whisker_high",
                                                         "Whisker.low": "whisker_low",
                                                         "Economy..GDP.per.Capita.": "economy_gdp",
                                                         "Family": "fam",
                                                         "Health..Life.Expectancy.": "health_life_expectancy",
                                                         "Freedom": "freedom",
                                                         "Generosity": "generosity",
                                                         "Trust..Government.Corruption.": "trust_gov",
                                                         "Dystopia.Residual": "dystopia"})

happiness_transformed.head()



,country,happiness_rank,happiness_score,whisker_high,whisker_low,economy_gdp,fam,health_life_expectancy,freedom,generosity,trust_gov,dystopia
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


# Extract starbucks CSV dataset into DataFrames

In [48]:
starbucks_df = pd.read_csv('starbucks.csv',)
starbucks_df.head()

,Country,Population,Numer of Starbucks,Starbucks per million inhabitants
0,United States,316128839,11851,37.49
1,China,1357380000,1462,1.08
2,Canada,35158304,1425,40.53
3,Japan,127338621,1052,8.26
4,United Kingdom,64097085,770,12.01


# Renaming the starbucks data columns

In [61]:
starbucks_cols =['Country', 'Population', 'Numer of Starbucks', 'Starbucks per million inhabitants']

starbucks_transformed= starbucks_df[starbucks_cols].copy()


starbucks_transformed = starbucks_transformed.rename(columns={"Country": "country",
                                                         "Population": "population",
                                                         "Numer of Starbucks": "numer_of_starbucks",
                                                         "Starbucks per million inhabitants": "starbucks_per_million"})

starbucks_transformed.head()



,country,population,numer_of_starbucks,starbucks_per_million
0,United States,316128839,11851,37.49
1,China,1357380000,1462,1.08
2,Canada,35158304,1425,40.53
3,Japan,127338621,1052,8.26
4,United Kingdom,64097085,770,12.01


# Connect to local postgresql database

In [52]:
from sqlalchemy import create_engine

In [62]:
connection_string = "postgres:postgres@localhost:5432/Starbucks_Happiness_DB"
engine = create_engine(f'postgresql://{connection_string}')

# Check for the tables created in postgresql

In [64]:
engine.table_names()

['starbucks', 'happiness', 'starbuckshappiness_final']

## Use pandas to clean and load happiness csv converted DataFrame into database

In [57]:
happiness_transformed.to_sql(name='happiness', con=engine, if_exists='append', index=False)


## Use pandas to clean and load starbucks csv converted Dataframe into database

In [65]:
starbucks_transformed.to_sql(name='starbucks', con=engine, if_exists='append', index=False)

# Confirm data has been added by querying the happiness table

In [66]:
pd.read_sql_query('select * from happiness', con=engine).head()

,country,happiness_rank,happiness_score,whisker_high,whisker_low,economy_gdp,fam,health_life_expectancy,freedom,generosity,trust_gov,dystopia
0,Norway,1,7.54,7.59,7.48,1.62,1.53,0.80,0.64,0.36,0.32,2.28
1,Denmark,2,7.52,7.58,7.46,1.48,1.55,0.79,0.63,0.36,0.40,2.31
2,Iceland,3,7.50,7.62,7.39,1.48,1.61,0.83,0.63,0.48,0.15,2.32
3,Switzerland,4,7.49,7.56,7.43,1.56,1.52,0.86,0.62,0.29,0.37,2.28
4,Finland,5,7.47,7.53,7.41,1.44,1.54,0.81,0.62,0.25,0.38,2.43


# confirm data has been added by querying the starbucks table 

In [67]:
pd.read_sql_query('select * from starbucks', con=engine).head()

,country,population,numer_of_starbucks,starbucks_per_million
0,United States,316128839,11851,37
1,China,1357380000,1462,1
2,Canada,35158304,1425,41
3,Japan,127338621,1052,8
4,United Kingdom,64097085,770,12


# Transformation of the two tables data was done on postgresql database and inserted into starbuckshappiness_final table